In [ ]:
''' O projeto se divide em análise de timesires e uma tentativa de prever evazão
fazendo uso de um algoritmo supervisionado
----Passos Time series
1 - Baixar o Log no moodle de eventos no moodle para atividades de participação -
OK
2 - Filtrar os usuário que não são alunos - Excel -pré processamento
OK
3 - Fazer discretização(IP) - Coluna Rede_Puc
OK
4 - Adicionar uma coluna referente ao cronograma(unidade 1 e etc)
Ok
5 - Adicionar coluna com as turmas
OK
6-  Plotar gráficos com a visualização
OK
------
------ Passos Forecasting
1 - Converter a tabela de time series em um data set anterior em um transicional
    1.1 - Plotar gráfico com estatśticas gerais de uso -OK
    1.2 - Talvez clusterizar os alunos - OK
2 - Descobrir quais usuários trancaram a disciplina - Log de atividades CLI
OK
3 - Descobrir se existe uma data com pico de trancamento
OK
4 - Baseado na relação entre unidade/trancamento definir um intervalo e filtrar o dataset antes da data de interesse.
(vai ser os usuários que trancaram durante a unidade 3 e unidade 4 )
OK 

5- Mesclar com o dataset dos usuários. Adicionar mais uma coluna(trancou[1,0])
OK
6 - Alimentar random forest
OK
7 - avaliar desempenho
OK
8 - extrair regras
'''
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from datetime import datetime        
%matplotlib inline
sns.set()

In [ ]:
#time series att
def rede_puc(turmas):
    ip = '139.82'
    rede = []
    for x in turmas.IP:
        if(ip == x[0:6]):
            rede.append('S')
        else:
            rede.append('N')
    turmas.insert(1, column='Rede_Puc', value=rede)#Passo 5
    return turmas

In [ ]:
def obter_turmas():
    df_ = pd.DataFrame()
    frames = [df_]
    
    for dados_turma in glob.glob('dados/turmas/*'):
        grupo = dados_turma[13:-4]# obtem código do grupo
        df = pd.read_csv(dados_turma)#lê arquivo
        df.insert(3, column='Turma', value=grupo)#insere coluna
        print(df.shape)
        frames.append(df)
    dados = pd.concat(frames)

    dados = normaliza_data(dados)
    
    
    return dados



In [ ]:
def normaliza_data(dados):
    dados.columns = [c.replace(' ', '_') for c in dados.columns]#Troca espaço por underline no cabeçalho
    dados.drop_duplicates(keep=False, inplace=True)#limpando dados duplicados
    dados['Data'], dados['Hora'] = dados['Hora'].str.split(' ', 1).str#divide a coluna Hora em duas colunas
    dados.Data = pd.to_datetime(dados.Data,dayfirst=True)
    return dados

In [ ]:
#time series att
def obter_unidades(t): 
    unidades= []
    data_unidades = {
        'IU1':datetime(2018, 3, 19),
        'FU1':datetime(2018, 4, 18),
        'IU2':datetime(2018, 4, 18),
        'FU2':datetime(2018, 5, 9),
        'IU3':datetime(2018, 5, 9),
        'FU3':datetime(2018, 5, 23),
        'IU4':datetime(2018, 5, 23),
        'FU4':datetime(2018, 6, 27)
    }

    for x in t.Data:
        if(data_unidades['IU1'] <= x < data_unidades['FU1']):
            unidades.append('unidade_1')
        elif(data_unidades['IU2'] <= x < data_unidades['FU2']):
            unidades.append('unidade_2')
        elif(data_unidades['IU3'] <= x < data_unidades['FU3']):
            unidades.append('unidade_3')
        elif(data_unidades['IU4'] <= x < data_unidades['FU4']):
            unidades.append('unidade_4')
        else:
            unidades.append('fora_de_epoca')
            
        
    t.insert(1, column='Unidade', value=unidades)
    return t

   

In [ ]:
def obter_periodo_dia(t):
    periodo = {
        'P1I':'00:00',
        'P1F':'04:00',
        'P2I':'04:00',
        'P2F':'08:00',
        'P3I':'08:00',
        'P3F':'12:00',
        'P4I':'12:00',
        'P4F':'16:00',
        'P5I':'16:00',
        'P5F':'20:00',
        'P6I':'20:00',
        'P6F':'24:00'
    }
    periodo_list = []
    for x in t.Hora:
        if (periodo['P1I'] <= x < periodo['P1F']):
            periodo_list.append('1')
        elif(periodo['P2I'] <= x < periodo['P2F']):
            periodo_list.append('2')
        elif(periodo['P3I'] <= x < periodo['P3F']):
            periodo_list.append('3')
        elif(periodo['P4I'] <= x < periodo['P4F']):
            periodo_list.append('4')
        elif(periodo['P5I'] <= x < periodo['P5F']):
            periodo_list.append('5')
        elif(periodo['P6I'] <= x < periodo['P6F']):
            periodo_list.append('6')

        
    t.insert(1, column='Periodo', value=periodo_list)
    return t

    

In [ ]:
turmas = obter_turmas() 
turmas = rede_puc(turmas)
turmas = obter_unidades(turmas)
turmas = obter_periodo_dia(turmas)




turmas.to_csv('turmas.csv')

In [ ]:

turmas['Unidade'].value_counts().sort_index().plot.bar(figsize=(24, 12));

In [ ]:
sns.countplot(turmas['Unidade']);

In [ ]:
turmas['Periodo'].value_counts().sort_index().plot.bar(figsize=(24, 12));

In [ ]:
sns.countplot(x=turmas['Periodo'],data=turmas);

In [ ]:
    recursos = turmas['Contexto_do_Evento'].value_counts().plot.bar(figsize=(24, 12));
recursos.set_title("Recursos", fontsize=20);
turmas['Contexto_do_Evento'].value_counts()


In [ ]:
turmas['Turma'].value_counts().sort_index().plot.bar(figsize=(24, 12));


In [ ]:
sns.countplot(x=turmas['Turma'],data=turmas);

In [ ]:
turmas['Rede_Puc'].value_counts().sort_index().plot.bar(figsize=(24, 12));


In [ ]:
#Parte 2
turmas_pre_clean=turmas.groupby('Nome_completo')['Contexto_do_Evento'].value_counts().sort_index().unstack().fillna(0)#tranformando timesires em data frame
turmas_pre_clean.head()

In [ ]:
turmas_pre_clean.describe()


In [ ]:
from sklearn.cluster import KMeans
from sklearn import decomposition
def cluster(t):
    data=t
    kmeans = KMeans(n_clusters = 8, init = 'k-means++')
    kmeans = kmeans.fit(data)
    plot_2D_cluster(data,kmeans)
    plot_3D_cluster(data,kmeans)
    return kmeans
    
    

In [ ]:
def plot_2D_cluster(data,kmeans):
    pca = decomposition.PCA()
    pca.n_components = 2
    X_reduced = pca.fit_transform(data)
    values = kmeans.cluster_centers_.squeeze()
    labels = kmeans.labels_
    centroid = pca.fit_transform(values)
    centroid_label = set(labels) 
    plt.scatter(X_reduced[:,0],X_reduced[:,1],c=labels,cmap='tab20')
    #plt.scatter(centroid[:,0],centroid[:,1],c=range(0,len(centroid_label)),cmap='Set1',marker='D')
    plt.show()
    

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def plot_3D_cluster(data,kmeans):
    fig = plt.figure()
    ax = Axes3D(fig)
    
    pca = decomposition.PCA()
    pca.n_components = 3
    X_reduced = pca.fit_transform(data)
    values = kmeans.cluster_centers_.squeeze()
    labels = kmeans.labels_
    centroid = pca.fit_transform(values)
    centroid_label = set(labels) 
    
    ax.scatter(X_reduced[:,0],X_reduced[:,1],X_reduced[:,1] , c=labels,cmap='tab20')
    #ax.scatter(centroid[:,0],centroid[:,1],centroid[:,2],c=range(0,len(centroid_label)),cmap='Set1',marker='D')
    plt.show()

In [ ]:
def obter_trancamentos():
    trancamento = pd.read_csv('dados/trancamento.csv')
    trancamento = normaliza_data(trancamento)
    filtro = trancamento[trancamento.Data < '2018-06-27 08:00:00']#filtra a data
    return filtro

In [ ]:
kmeans = cluster(turmas_pre_clean.iloc[:, :,].values)
#3-5-8

In [ ]:
turmas_pre_clean.to_csv('alunos_clean.csv')

In [ ]:
alunos_clean = pd.read_csv('alunos_clean.csv')
alunos_clean.head(3)

In [ ]:
from slugify import slugify
def obter_log_evadidos (evasao):
    # Remove usuários que trancaram a disciplina no periodo fora de interesse
    target = evasao[(evasao.Unidade == 'unidade_3') | (evasao.Unidade == 'unidade_4')]
    target.sort_values(by='Data')
    id_alunos = []
    for i in target['Evento']:
        id_alunos.append(i[-36:-31])#usados na query
        
    alunos_evadidos =  pd.read_csv('dados/trancamento_clean.csv')
 
    return alunos_evadidos
    

In [ ]:
describe = alunos_clean.describe()
describe.to_csv('metricas.csv')

In [ ]:
evasao = obter_trancamentos()
evasao = obter_unidades(evasao)
evasao.head(3)

In [ ]:
evasao['Unidade'].value_counts().sort_index().plot.bar(figsize=(24, 12));

In [ ]:

log_evadidos = obter_log_evadidos(evasao)

log_evadidos.head(3)



In [ ]:
log_evadidos['component'].value_counts().sort_index().plot.bar(figsize=(24, 12));

In [ ]:
evadidos_data=log_evadidos.groupby('userid')['component'].value_counts().sort_index().unstack().fillna(0)#tranformando timesires em data frame
evadidos_data.to_csv('evadidos_count.csv')
evadidos_data = pd.read_csv('evadidos_count.csv')
print(len(evadidos_data))
print(len(alunos_clean))

In [ ]:
def merge_alunos(evadidos, alunos_clean):
    #evadidos.rename(columns={'userid':'Nome_completo'}, inplace=True)
    evadidos.drop(['userid'], axis=1)
    alunos_clean.drop(['Nome_completo'], axis=1)
    
    alunos_clean['Trancou'] = 0
    evadidos['Trancou'] = 1
    return pd.concat([alunos_clean, evadidos_data],join="inner",sort=True,ignore_index=True)
    

    
    

In [ ]:
evadidos_data.head()
alunos_final = merge_alunos(evadidos_data, alunos_clean)
alunos_final.head()

In [ ]:
dados_finais = alunos_final.drop(['Trancou'], axis=1)
targets_finais = alunos_final['Trancou']
targets_finais

In [ ]:
cluster(dados_finais.values);

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dados_finais.values, 
 targets_finais.values, 
 test_size = 0.4, 
 random_state = 0)
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))
print(y_train)
print(y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
rf = RandomForestClassifier(random_state=0, n_estimators = 200, class_weight="balanced")
model = rf.fit(X_train, y_train)


In [ ]:
resultado = rf.predict(X_test)
resultado

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score


precision, recall, fscore, support = score(y_test, resultado)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))


In [ ]:
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
feature_list = list(dados_finais.columns)
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree3.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree3.dot')
# Write graph to a png file
#graph.write_png('tree.png')

In [ ]:
graph.write_png('tree.png')


In [ ]:
dados_finais.columns


 